# Análisis de datos utilizando PyRoot


###Cargo los paquetes nesesarias

In [1]:
#from IPython.core.interactiveshell import InteractiveShell
import ROOT
import os
import commands
#InteractiveShell.ast_node_interactivity = "all" # Para tener el modo interactivo completo

###Cargo las Librerías nesesarias

In [2]:
ROOT.gROOT.Reset()
#Esta es la ruta a la librería dinámica de Delphes (debe de ser compilada con la misma versión de root del notebook)
#ROOT.gSystem.AddDynamicPath("/scratch/cms/MadGraph/MG5_aMC_v2_5_2/Delphes/")
ROOT.gSystem.AddDynamicPath("/scratch/cms/Delphes/delphes/")
ROOT.gSystem.Load("libDelphes");

###Construcción de los TChain

In [3]:
#Lista de carpetas le analisis
Path1= "/scratch/cms/Datos"
Path2= "/Events"
Path3= "/tag_1_delphes_events.root"
ListOfFolders = ["/BackGround-DY_2j-Run0","/BackGround-tW-Run0","/BackGround-WW-Run0","/Signal_2j_mu-nu-Run0","/BackGround-mumuW"]


In [4]:
#Para guardar la info sobre los cortes
signal={};
for Folder in ListOfFolders:
    comand ="ls " + Path1 + Folder + Path2;
    tmp = commands.getstatusoutput( comand )
    
    #Parto la cadena de caracteres por '\n', y asigno las partes a un arreglo
    Runs = tmp[1].split("\n");
    
    #Creo el objeto TChain
    MainChain=ROOT.TChain("Delphes")
    
    #Lleno el objeto MainChain con las salidas de MadGraph-Pythia_Delphes
    for run in range(len(Runs)):
        imput = Path1 + Folder + Path2 + "/" + Runs[run] + "/tag_1_delphes_events.root"
        MainChain.Add(imput)
        
    #Creo el objeto ExRootTreeReader
    treeReader = ROOT.ExRootTreeReader(MainChain)
    numberOfEntries = treeReader.GetEntries()
    
    #Obtengo las ramas que voy a usar
    branchJet = treeReader.UseBranch("Jet")
    branchMuon = treeReader.UseBranch("Muon")
    branchMissingET = treeReader.UseBranch("MissingET")

    # Inicializo los contadores
    cut1=0; cut2=0; cut3=0; cut4=0; cut5=0;
    cuts={};
    
    # Loop sobre los eventos
    for entry in range(0, numberOfEntries):
        #Se carga la entrada espesífica
        treeReader.ReadEntry(entry)
        
        # Primer corte, al menos 2 jet
        if branchJet.GetEntries() > 1:
            cut1 = cut1 +1;
            bcount=0;
            for i in range(0, branchJet.GetEntries() ): #Esta parte es la que produce el error (TClass::TClass:0: RuntimeWarning: no dictionary for class CompBase is available)
                jet = branchJet.At(i)
                #Cuento cuantos jet hay identificados como b quarks
                if jet.BTag :
                    bcount = bcount + 1;
            
            # Segundo corte, al menos 2 jet b       
            if bcount > 1:
                cut2 = cut2 +1;
            
                # If event contains at least 2 muons
                if branchMuon.GetEntries() > 1:
                    cut3 = cut3 +1;
                    # Tomo los dos muones mas energéticos y comparo sus cargas eléctricas
                    muon1 = branchMuon.At(0)
                    muon2 = branchMuon.At(1)
                
                    #Corte 4: Que los muones mas energéticos tengan cargas diferentes
                    if muon1.Charge*muon2.Charge < 0:
                        cut4 = cut4 +1;

                    
                        #Corte 5: muones mas energéticos tengan masa invariante diferente de la del Z
                        MassMuon= (muon1.P4()+muon2.P4()).M();
                        if not((MassMuon>80) and (MassMuon<100)):
                            cut5 = cut5 +1;
    
    
    cuts['numberOfEntries']=numberOfEntries;
    cuts['cut1'] = cut1;
    cuts['cut2'] = cut2;
    cuts['cut3'] = cut3;
    cuts['cut4'] = cut4;
    cuts['cut5'] = cut5; 
    signal[Folder]=cuts

TClass::TClass:0: RuntimeWarning: no dictionary for class CompBase is available


In [5]:
signal

{'/BackGround-DY_2j-Run0': {'cut1': 28247,
  'cut2': 317,
  'cut3': 137,
  'cut4': 137,
  'cut5': 14,
  'numberOfEntries': 160448L},
 '/BackGround-WW-Run0': {'cut1': 74134,
  'cut2': 25633,
  'cut3': 11387,
  'cut4': 11387,
  'cut5': 9838,
  'numberOfEntries': 85553L},
 '/BackGround-mumuW': {'cut1': 238907,
  'cut2': 511,
  'cut3': 196,
  'cut4': 196,
  'cut5': 10,
  'numberOfEntries': 325964L},
 '/BackGround-tW-Run0': {'cut1': 76727,
  'cut2': 27754,
  'cut3': 12751,
  'cut4': 12750,
  'cut5': 10985,
  'numberOfEntries': 80417L},
 '/Signal_2j_mu-nu-Run0': {'cut1': 341827,
  'cut2': 126190,
  'cut3': 55688,
  'cut4': 55687,
  'cut5': 48077,
  'numberOfEntries': 355148L}}

In [10]:
EffCut={};
Eff={};
for entry in signal.keys():
    for i in signal[entry].keys():
        EffCut[i] = (0.1*signal[entry][i])/(0.1*signal[entry]['numberOfEntries']);
    Eff[entry] = EffCut;

In [11]:
Eff

{'/BackGround-DY_2j-Run0': {'cut1': 0.8665271819807604,
  'cut2': 0.29961544305868876,
  'cut3': 0.1330987808726754,
  'cut4': 0.1330987808726754,
  'cut5': 0.11499304524680606,
  'numberOfEntries': 1.0},
 '/BackGround-WW-Run0': {'cut1': 0.8665271819807604,
  'cut2': 0.29961544305868876,
  'cut3': 0.1330987808726754,
  'cut4': 0.1330987808726754,
  'cut5': 0.11499304524680606,
  'numberOfEntries': 1.0},
 '/BackGround-mumuW': {'cut1': 0.8665271819807604,
  'cut2': 0.29961544305868876,
  'cut3': 0.1330987808726754,
  'cut4': 0.1330987808726754,
  'cut5': 0.11499304524680606,
  'numberOfEntries': 1.0},
 '/BackGround-tW-Run0': {'cut1': 0.8665271819807604,
  'cut2': 0.29961544305868876,
  'cut3': 0.1330987808726754,
  'cut4': 0.1330987808726754,
  'cut5': 0.11499304524680606,
  'numberOfEntries': 1.0},
 '/Signal_2j_mu-nu-Run0': {'cut1': 0.8665271819807604,
  'cut2': 0.29961544305868876,
  'cut3': 0.1330987808726754,
  'cut4': 0.1330987808726754,
  'cut5': 0.11499304524680606,
  'numberOfEnt

In [ ]:
Eff.